In [1]:
# Import Libraires and tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_spring_2023 import acweSaveSeg_v5, acweConfidenceMapTools_v3
from DatasetTools import DataManagmentTools as dmt

In [2]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

In [3]:
# Key Varibles

# Carrington Rotation
cr = 'CR2099'

# Dataset Names
traceFolder = '/home/jgra/Coronal Holes/Code VI/ACWEcorrectionTest/DatasetTools/DownloadLists/'
dataset = '/home/jgra/Coronal Holes/newDataset/'
conMapFolder = '/mnt/coronal_holes/CodeVI Observations/ConMapStandardDefault_5_1/'

# ACWE Parameters 
acweChoice = '193'

# Data Summary
datasetFile = 'Results/' + cr + '.GrowthAndIntensity.npz'

In [4]:
# Open and find key data

# Load Data Summary
data = np.load(datasetFile, allow_pickle=True)
lst = data.files

seedArea     = data[lst[0]]
seedIntnMean = data[lst[1]]
seedIntnStd  = data[lst[2]]
seedIntnMin  = data[lst[3]]
seedIntnMax  = data[lst[4]]
segArea      = data[lst[5]]
segIntnMean  = data[lst[6]]
segIntnStd   = data[lst[7]]
segIntnMin   = data[lst[8]]
segIntnMax   = data[lst[9]]
bkgWeights   = data[lst[10]]
IOO          = data[lst[11]]

# Open Carrington Rotation Document
CarringtonFile = traceFolder + cr + '.csv'
data = pd.read_csv(CarringtonFile,header=0)
keys = data.keys()

conMaps = sorted(glob.glob(conMapFolder + cr + '/*/*.npz'))

In [5]:
# Ensure Folder Exists
saveDirectory = '/Figures/ConMaps/ChangeOfTargetAmbiguity/'
saveDirectory = ROOT_DIR + saveDirectory

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)

In [ ]:
#collection = [74,75,76,77,456,459,464,479]
collection = [479,520]

# Cycle Through Special Collection
for i in collection:
    
    # Get File Name
    file = data[acweChoice][i]
    
    # Get File Times
    time = dmt.timeFormat(dmt.timeFromFilenameAIA(file))
    
    # Area and Mean Stats
    normalizedSegArea    = segArea[i]     / (seedArea[i].astype(float))
    normalizedSegMean    = segIntnMean[i] / (seedIntnMean[i].astype(float))
    normalizedSegMax     = segIntnMax[i]  / (seedIntnMean[i].astype(float))
    normalizedSegMin     = segIntnMin[i]  / (seedIntnMean[i].astype(float))
    normalizedSeedMin    = np.empty(len(normalizedSegMin))
    normalizedSeedMin[:] = seedIntnMin[i] / (seedIntnMean[i].astype(float))
    normalizedSeedMax    = np.empty(len(normalizedSeedMin))
    normalizedSeedMax[:] = seedIntnMax[i] / (seedIntnMean[i].astype(float))
    
    # Open Map
    I,Idsp,H = openAIA(dataset+cr+'/'+file)
    
    # Open Old Confidence Maps
    H,AH,SEG = acweSaveSeg_v5.openSeg(conMaps[i])
    
    # Extract inital Mask
    MASK = AH['INIT_MASK']
    
    # Standard Confidence Map
    conMap = np.sum(SEG,axis=0).astype(float)/len(SEG)
    
    # "Smart" Confidence Map
    conMapSmart = acweConfidenceMapTools_v3.smartConMap(SEG,AH,restoreScale=False)
    
    # Plot Results
    plt.figure(figsize=[55,20])
    #plt.figure(figsize=[110,20])
    plt.rcParams.update({'font.size': 60})
    plt.suptitle(time)
    #plt.rcParams.update({'font.size': 50})
    plt.subplot(1,2,1)
    plt.imshow(np.flip(Idsp,axis=0),cmap ='gray')
    plt.title('Observation')
    plt.axis(False)
    plt.subplot(1,2,2)
    plt.imshow(np.flip(MASK,axis=0),interpolation='None')
    plt.axis(False)
    plt.title('Initial Seed')
    
    # Save and Show Figure
    figTitle = saveDirectory + 'Seed.' + os.path.basename(file) + '.png'
    plt.savefig(figTitle)
    plt.show()
    
    plt.figure(figsize=[55,20])
    plt.rcParams.update({'font.size': 60})
    plt.suptitle(time)
    plt.rcParams.update({'font.size': 50})
    plt.subplot(1,2,1)
    plt.imshow(np.flip(conMap,axis=0))
    plt.colorbar()
    plt.axis(False)
    plt.title('Full Segmentation Group')
    plt.subplot(1,2,2)
    plt.imshow(np.flip(conMapSmart,axis=0))
    plt.colorbar()
    plt.axis(False)
    plt.title('Smart Confidence Map')
    
    # Save and Show Figure
    figTitle = saveDirectory + 'Maps.' + os.path.basename(file) + '.png'
    plt.savefig(figTitle)
    plt.show()
    
    # Intensity Behaviour
    plt.figure(figsize=[50,12])
    plt.rcParams.update({'font.size': 40})
    plt.plot((1.0)/bkgWeights[i],normalizedSegMean,label='Mean')
    plt.plot((1.0)/bkgWeights[i],normalizedSegMax,label='Max')
    plt.plot((1.0)/bkgWeights[i],normalizedSegMin,label='Min')
    plt.plot((1.0)/bkgWeights[i],normalizedSeedMin,':',label='Seed Min Bound')
    plt.plot((1.0)/bkgWeights[i],normalizedSeedMax,':',label='Seed Max Bound')
    plt.legend()
    title = time + '\nIntensity of Segmentation as a function of $\lambda_i/\lambda_o$ ratio'
    plt.title(title)
    xlabel = '$\lambda_i/\lambda_o$'
    plt.xlabel(xlabel)
    plt.ylabel('Intensity/mean(Seed)')
    plt.grid()
    
    # Save and Show Figure
    figTitle = saveDirectory + 'IntensityStats.' + os.path.basename(file) + '.eps'
    plt.savefig(figTitle)
    plt.show()
    
    # Percent of seed Retained
    plt.figure(figsize=[50,13])
    plt.rcParams.update({'font.size': 44})
    plt.plot(1/bkgWeights[i],IOO[i])
    title = time + '\nPercent of Inital Seed Retained as a function of $\lambda_i/\lambda_o$ ratio'
    plt.title(title)
    xlabel = '$\lambda_i/\lambda_o$'
    plt.xlabel(xlabel)
    plt.ylabel('Percent of Seed Retained')
    plt.grid()
    plt.ylim([-0.05,1.05])
    plt.xlim([5,105])
    
    # Save and Show Figure
    figTitle = saveDirectory + 'SeedRetainedStats.' + os.path.basename(file) + '.eps'
    plt.savefig(figTitle)
    plt.show()